<b>Train FastText embeddings using gensim</b> (Source: https://radimrehurek.com/gensim/auto_examples/tutorials/run_fasttext.html)

In [1]:
from pprint import pprint as print
from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors

Load corpus & Preprocess raw text (Remove punctuation and numbers) (ca. 1.5h per corpus)

In [ ]:
import spacy

nlp = spacy.load("de_core_news_sm")

new_corpus = []
#i=1
with open('raw_data/deu_news_2020_1M-sentences.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        doc = nlp(line)
        new_line = [token.lower_ for token in doc if token.is_alpha]
        for j in range(10):
            new_line.append('$')
        new_corpus.append(' '.join(new_line))
        #i+=1
        #if i>200:
            #break 
    f.close()

with open('clean_data/deu_news_2020_clean.txt', 'w', encoding='utf-8') as f_out:
    for line in new_corpus:
        f_out.write(line)
        f_out.write('\n')
    f_out.close()

Train model (window=5 10 min., 3 -> 6 min., 10 -> 12 min.)

In [ ]:
model_1995 = FastText(sg=1, vector_size=300, window=10)
corpus_file_1995 = 'clean_data/deu_news_1995_clean.txt'

# build the vocabulary
model_1995.build_vocab(corpus_file=corpus_file_1995)

# train the model
model_1995.train(
    corpus_file=corpus_file_1995, epochs=model_1995.epochs,
    total_examples=model_1995.corpus_count, total_words=model_1995.corpus_total_words,
)

print(model_1995)

Word vector lookup

In [ ]:
wv_1995 = model_1995.wv
print(wv_1995)

Load other corpus & build models

In [ ]:
corpus_file_2010 = 'clean_data/deu_news_2010_clean.txt'
model_2010 = FastText(sg=1, vector_size=300)

corpus_file_2020 = 'clean_data/deu_news_2020_clean.txt'
model_2020 = FastText(sg=1, vector_size=300)

In [ ]:
# build vocabularies
model_2010.build_vocab(corpus_file=corpus_file_2010)
model_2020.build_vocab(corpus_file=corpus_file_2020)

# train models
model_2010.train(
    corpus_file=corpus_file_2010, epochs=model_2010.epochs,
    total_examples=model_2010.corpus_count, total_words=model_2010.corpus_total_words,
)
model_2020.train(
    corpus_file=corpus_file_2020, epochs=model_2020.epochs,
    total_examples=model_2020.corpus_count, total_words=model_2020.corpus_total_words,
)

In [ ]:
wv_2010 = model_2010.wv
wv_2020 = model_2020.wv

Save words & their embeddings (once training is complete)

In [ ]:
#wv_1995.save("trained_models/model_1995_training_3_wordvectors")
#wv_2010.save("trained_models/model_2010_wordvectors")
#wv_2020.save("trained_models/model_2020_wordvectors")

Load models (2-6 min.)

In [ ]:
wv_1995 = KeyedVectors.load("trained_models/model_1995_training_3_wordvectors", mmap='r')
wv_2010 = KeyedVectors.load("trained_models/model_2010_training_3_wordvectors", mmap='r')
wv_2020 = KeyedVectors.load("trained_models/model_2020_training_3_wordvectors", mmap='r')

Inspect some semantic similarities

In [ ]:
print(wv_1995.similarity("mann", "arzt"))
print(wv_1995.similarity("frau", "arzt"))
print(wv_1995.similarity("frau", "ärztin"))
print(wv_1995.similarity("frau", "krankenschwester"))
print(wv_1995.similarity("mann", "krankenschwester"))
print(wv_1995.similarity("mann", "krankenpfleger"))

In [ ]:
print(wv_2010.similarity("mann", "arzt"))
print(wv_2010.similarity("frau", "arzt"))
print(wv_2010.similarity("frau", "ärztin"))
print(wv_2010.similarity("frau", "krankenschwester"))
print(wv_2010.similarity("mann", "krankenschwester"))
print(wv_2010.similarity("mann", "krankenpfleger"))

In [ ]:
print(wv_2020.similarity("mann", "arzt"))
print(wv_2020.similarity("frau", "arzt"))
print(wv_2020.similarity("frau", "ärztin"))
print(wv_2020.similarity("frau", "krankenschwester"))
print(wv_2020.similarity("mann", "krankenschwester"))
print(wv_2020.similarity("mann", "krankenpfleger"))

Print words with most similar vector

In [ ]:
print(wv_1995.most_similar("arzt"))
print(wv_2010.most_similar("arzt"))
print(wv_2020.most_similar("arzt"))

Example analogy set

In [ ]:
print(wv_1995.most_similar(positive=['frau', 'arzt'], negative=['mann']))
print(wv_2010.most_similar(positive=['frau', 'arzt'], negative=['mann']))
print(wv_2020.most_similar(positive=['frau', 'arzt'], negative=['mann']))

Visualization with PCA (Source: https://machinelearningmastery.com/develop-word-embeddings-python-gensim/) > 12 min. 

In [ ]:
from sklearn.decomposition import PCA
from matplotlib import pyplot

In [ ]:
# fit a 2d PCA model to the vectors
X = wv_1995
pca = PCA(n_components=2)
result = pca.fit_transform(X)
# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(wv_1995[:20])
for i, word in enumerate(words):
	pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

Source: https://web.stanford.edu/class/cs224n/materials/Gensim%20word%20vector%20visualization.html

In [ ]:
"""import numpy as np

%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def display_pca_scatterplot(model, words=None, sample=0):
    if words == None:
        if sample > 0:
            words = np.random.choice(list(model.vocab.keys()), sample)
        else:
            words = [ word for word in model.vocab ]
        
    word_vectors = np.array([model[w] for w in words])

    twodim = PCA().fit_transform(word_vectors)[:,:2]
    
    plt.figure(figsize=(6,6))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    for word, (x,y) in zip(words, twodim):
        plt.text(x+0.05, y+0.05, word)

display_pca_scatterplot(model, sample=300)

# display_pca_scatterplot(model, ['frau', 'mann', 'arzt', 'ärztin'])"""

Evaluate general performance on semantic German analogy set, Source: https://devmount.github.io/GermanWordEmbeddings/ 

In [ ]:
analogies_result = wv_1995.evaluate_word_analogies('questions/semantic_evaluation.txt')
print(analogies_result)

In [ ]:
analogies_result = wv_2010.evaluate_word_analogies('questions/semantic_evaluation.txt')
print(analogies_result)

In [ ]:
analogies_result = wv_2020.evaluate_word_analogies('questions/semantic_evaluation.txt')
print(analogies_result)

Gensim English question set

In [ ]:
#analogies_result = wv_2020.evaluate_word_analogies('questions/gensim questions-words.txt')
#print(analogies_result)

<b>Word Embedding Association Test (WEAT) by Chaloner & Maldonado (2019)</b>

Bias categories:

B1: career vs family
B2: maths vs arts 
B3: science vs arts 
B4: intelligence vs appearance
B5: strength vs weakness

Both groups of target words per category are compared to the two attribute sets female and male

Train models and save word vectors with gensim 4.0.1 (10 min. per model)

In [2]:
corpus_file_1995 = 'clean_data/deu_news_1995_clean.txt'
model_1995 = FastText(sg=1, size=300, window=10)
model_1995.build_vocab(corpus_file=corpus_file_1995)
model_1995.train(
    corpus_file=corpus_file_1995, epochs=model_1995.epochs,
    total_examples=model_1995.corpus_count, total_words=model_1995.corpus_total_words,
)
wv_1995 = model_1995.wv

In [3]:
corpus_file_2010 = 'clean_data/deu_news_2010_clean.txt'
model_2010 = FastText(sg=1, size=300, window=10)
model_2010.build_vocab(corpus_file=corpus_file_2010)
model_2010.train(
    corpus_file=corpus_file_2010, epochs=model_2010.epochs,
    total_examples=model_2010.corpus_count, total_words=model_2010.corpus_total_words,
)
wv_2010 = model_2010.wv

In [4]:
corpus_file_2020 = 'clean_data/deu_news_2020_clean.txt'
model_2020 = FastText(sg=1, size=300, window=10)
model_2020.build_vocab(corpus_file=corpus_file_2020)
model_2020.train(
    corpus_file=corpus_file_2020, epochs=model_2020.epochs,
    total_examples=model_2020.corpus_count, total_words=model_2020.corpus_total_words,
)
wv_2020 = model_2020.wv

Save old version keyed vectors

In [5]:
wv_1995.save("gensim4.0.1_ft_models/vectors_1995.kv")
wv_2010.save("gensim4.0.1_ft_models/vectors_2010.kv")
wv_2020.save("gensim4.0.1_ft_models/vectors_2020.kv")

Load old version keyed vectors

In [ ]:
wv_1995 = KeyedVectors.load("gensim4.0.1_ft_models/vectors_1995.kv", mmap='r')
wv_2010 = KeyedVectors.load("gensim4.0.1_ft_models/vectors_2010.kv", mmap='r')
wv_2020 = KeyedVectors.load("gensim4.0.1_ft_models/vectors_2020.kv", mmap='r')

Exploration with Responsibly (https://docs.responsibly.ai/)

In [6]:
from responsibly.we.utils import most_similar, cosine_similarities_by_words

Compute most similar words without restriction (words from sets may be repeated)

In [7]:
print(most_similar(wv_1995, positive=['frau', 'arzt'], negative=['mann']))
print(most_similar(wv_2010, positive=['frau', 'arzt'], negative=['mann']))
print(most_similar(wv_2020, positive=['frau', 'arzt'], negative=['mann']))

[('arzt', 0.7294990356421993),
 ('patientin', 0.6411809819084028),
 ('ärztin', 0.6019659485813017),
 ('hausarzt', 0.5998664150539841),
 ('physiotherapeutin', 0.579768095891821),
 ('therapeutin', 0.5710132638183131),
 ('hautarzt', 0.5633543766755207),
 ('medizinerin', 0.561739535056716),
 ('arzthelferin', 0.5613126716369358),
 ('patient', 0.5597132866626509)]
[('arzt', 0.7453792053776191),
 ('frauenarzt', 0.6158223903324075),
 ('hausarzt', 0.5977107747931595),
 ('patientin', 0.5826124837682605),
 ('zahnarzt', 0.5791214804080856),
 ('ärztin', 0.574488448786975),
 ('medizinerin', 0.5728439805654315),
 ('tierarzt', 0.562282196262649),
 ('frau', 0.5553649950714564),
 ('arztberuf', 0.5454157208356099)]
[('arzt', 0.7787683393165297),
 ('ärztin', 0.6663402508733298),
 ('hausarzt', 0.6365236662554131),
 ('frauenarzt', 0.6265268355491721),
 ('hausärztin', 0.6176358832638997),
 ('zahnärztin', 0.6144357211383189),
 ('zahnarzt', 0.6106656290968133),
 ('amtsarzt', 0.6020358630467473),
 ('amtsärztin'

In [8]:
sample_occupation_list = ['arzt','ärztin','krankenschwester','krankenpfleger']
print(cosine_similarities_by_words(wv_2020, 'frau', sample_occupation_list))
print(cosine_similarities_by_words(wv_2020, 'mann', sample_occupation_list))

array([0.26169273, 0.44548863, 0.46253282, 0.335657  ], dtype=float32)
array([0.30543205, 0.27106968, 0.39263615, 0.30068994], dtype=float32)


Read translated wordlists

<b>TODO</b> write function to load JSON?

In [21]:
import json

with open('WEAT_german/wissenschaft.json', encoding='utf-8') as data:
    wis_dict = json.load(data)
    data.close()

with open('WEAT_german/mathematik.json', encoding='utf-8') as data:
    mat_dict = json.load(data)
    data.close()

with open('WEAT_german/kunst.json', encoding='utf-8') as data:
    kunst_dict = json.load(data)
    data.close()

with open('WEAT_german/stark.json', encoding='utf-8') as data:
    stark_dict = json.load(data)
    data.close()

with open('WEAT_german/schwach.json', encoding='utf-8') as data:
    schwach_dict = json.load(data)
    data.close()

with open('WEAT_german/intelligenz.json', encoding='utf-8') as data:
    int_dict = json.load(data)
    data.close()

with open('WEAT_german/aussehen.json', encoding='utf-8') as data:
    aus_dict = json.load(data)
    data.close()

with open('WEAT_german/beruf.json', encoding='utf-8') as data:
    beruf_dict = json.load(data)
    data.close()

with open('WEAT_german/familie.json', encoding='utf-8') as data:
    fam_dict = json.load(data)
    data.close()

with open('WEAT_german/weiblich.json', encoding='utf-8') as data:
    w_dict = json.load(data)
    data.close()

with open('WEAT_german/maennlich.json', encoding='utf-8') as data:
    m_dict = json.load(data)
    data.close()

Implement WEAT

In [10]:
from responsibly.we.weat import calc_single_weat

B1: career vs. family

In [11]:
print(calc_single_weat(model=wv_1995, 
                    first_target=beruf_dict, 
                    second_target=fam_dict, 
                    first_attribute=w_dict, 
                    second_attribute=m_dict))

print(calc_single_weat(model=wv_2010, 
                    first_target=beruf_dict,  
                    second_target=fam_dict, 
                    first_attribute=w_dict, 
                    second_attribute=m_dict))

print(calc_single_weat(model=wv_2020, 
                    first_target=beruf_dict, 
                    second_target=fam_dict, 
                    first_attribute=w_dict, 
                    second_attribute=m_dict))

{'Attrib. words': 'weiblich vs. männlich',
 'Na': '11x2',
 'Nt': '8x2',
 'Target words': 'beruf vs. familie',
 'd': -1.3503735,
 'p': 0.9978243978243978,
 's': -0.5250897109508514}
{'Attrib. words': 'weiblich vs. männlich',
 'Na': '11x2',
 'Nt': '8x2',
 'Target words': 'beruf vs. familie',
 'd': -1.3187135,
 'p': 0.9965811965811966,
 's': -0.5579641908407211}
{'Attrib. words': 'weiblich vs. männlich',
 'Na': '11x2',
 'Nt': '8x2',
 'Target words': 'beruf vs. familie',
 'd': -1.3172334,
 'p': 0.9975135975135975,
 's': -0.5422469526529312}


B2: maths vs. arts

In [12]:
print(calc_single_weat(model=wv_1995, 
                    first_target=mat_dict, 
                    second_target=kunst_dict, 
                    first_attribute=w_dict, 
                    second_attribute=m_dict))

print(calc_single_weat(model=wv_2010, 
                    first_target=mat_dict,  
                    second_target=kunst_dict, 
                    first_attribute=w_dict, 
                    second_attribute=m_dict))

print(calc_single_weat(model=wv_2020, 
                    first_target=mat_dict, 
                    second_target=kunst_dict, 
                    first_attribute=w_dict, 
                    second_attribute=m_dict))

{'Attrib. words': 'weiblich vs. männlich',
 'Na': '11x2',
 'Nt': '8x2',
 'Target words': 'mathematik vs. kunst',
 'd': 1.1648192,
 'p': 0.00909090909090909,
 's': 0.2651319205760956}
{'Attrib. words': 'weiblich vs. männlich',
 'Na': '11x2',
 'Nt': '8x2',
 'Target words': 'mathematik vs. kunst',
 'd': 0.8342018,
 'p': 0.054234654234654234,
 's': 0.19725701212882996}
{'Attrib. words': 'weiblich vs. männlich',
 'Na': '11x2',
 'Nt': '8x2',
 'Target words': 'mathematik vs. kunst',
 'd': 0.3978146,
 'p': 0.2376068376068376,
 's': 0.08596093952655792}


B3: science vs arts

In [13]:
print(calc_single_weat(model=wv_1995, 
                    first_target=wis_dict, 
                    second_target=kunst_dict, 
                    first_attribute=w_dict, 
                    second_attribute=m_dict))

print(calc_single_weat(model=wv_2010, 
                    first_target=wis_dict,  
                    second_target=kunst_dict, 
                    first_attribute=w_dict, 
                    second_attribute=m_dict))

print(calc_single_weat(model=wv_2020, 
                    first_target=wis_dict, 
                    second_target=kunst_dict, 
                    first_attribute=w_dict, 
                    second_attribute=m_dict))

{'Attrib. words': 'weiblich vs. männlich',
 'Na': '11x2',
 'Nt': '8x2',
 'Target words': 'wissenschaft vs. kunst',
 'd': 0.09958489,
 'p': 0.4271950271950272,
 's': 0.021453166555147618}
{'Attrib. words': 'weiblich vs. männlich',
 'Na': '11x2',
 'Nt': '8x2',
 'Target words': 'wissenschaft vs. kunst',
 'd': -0.15480079,
 'p': 0.6131313131313131,
 's': -0.03475114703178406}
{'Attrib. words': 'weiblich vs. männlich',
 'Na': '11x2',
 'Nt': '8x2',
 'Target words': 'wissenschaft vs. kunst',
 'd': -0.70312375,
 'p': 0.9051282051282051,
 's': -0.21315891668200493}


B4: intelligence vs. appearance

In [22]:
print(calc_single_weat(model=wv_1995, 
                    first_target=int_dict, 
                    second_target=aus_dict, 
                    first_attribute=w_dict, 
                    second_attribute=m_dict))

print(calc_single_weat(model=wv_2010, 
                    first_target=int_dict,  
                    second_target=aus_dict, 
                    first_attribute=w_dict, 
                    second_attribute=m_dict))

print(calc_single_weat(model=wv_2020, 
                    first_target=int_dict, 
                    second_target=aus_dict, 
                    first_attribute=w_dict, 
                    second_attribute=m_dict))

{'Attrib. words': 'weiblich vs. männlich',
 'Na': '11x2',
 'Nt': '11x2',
 'Target words': 'intelligenz vs. aussehen',
 'd': -0.4827857,
 'p': 0.8583959899749374,
 's': -0.13892878592014313}
{'Attrib. words': 'weiblich vs. männlich',
 'Na': '11x2',
 'Nt': '11x2',
 'Target words': 'intelligenz vs. aussehen',
 'd': -0.26572827,
 'p': 0.7227117567674843,
 's': -0.06973184645175934}
{'Attrib. words': 'weiblich vs. männlich',
 'Na': '11x2',
 'Nt': '11x2',
 'Target words': 'intelligenz vs. aussehen',
 'd': 0.07158573,
 'p': 0.43606612685560053,
 's': 0.021325059235095978}


B5: strength vs. weakness

In [23]:
print(calc_single_weat(model=wv_1995, 
                    first_target=stark_dict, 
                    second_target=schwach_dict, 
                    first_attribute=w_dict, 
                    second_attribute=m_dict))

print(calc_single_weat(model=wv_2010, 
                    first_target=stark_dict,  
                    second_target=schwach_dict, 
                    first_attribute=w_dict, 
                    second_attribute=m_dict))

print(calc_single_weat(model=wv_2020, 
                    first_target=stark_dict, 
                    second_target=schwach_dict, 
                    first_attribute=w_dict, 
                    second_attribute=m_dict))

{'Attrib. words': 'weiblich vs. männlich',
 'Na': '11x2',
 'Nt': '11x2',
 'Target words': 'stark vs. schwach',
 'd': -0.31690106,
 'p': 0.7578916181857358,
 's': -0.08451436460018158}
{'Attrib. words': 'weiblich vs. männlich',
 'Na': '11x2',
 'Nt': '11x2',
 'Target words': 'stark vs. schwach',
 'd': -0.5758694,
 'p': 0.9019593667426484,
 's': -0.16809803247451782}
{'Attrib. words': 'weiblich vs. männlich',
 'Na': '11x2',
 'Nt': '11x2',
 'Target words': 'stark vs. schwach',
 'd': -0.6690417,
 'p': 0.936414849340546,
 's': -0.17794756591320038}


In [ ]:
#TODO ? Gonen & Goldberg
plot_most_biased_clustering(biased, debiased, seed='ends', n_extreme=500, random_state=1)